In [ ]:
# NOT quantum - machine learning classification code using pytorch

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define your data preprocessing functions
def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    return np.array([-beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2], dtype=np.float32)

# Load your data
data = np.loadtxt("trainX.txt")
X = data[:, 0:2]

# Apply angle encoding
features = [get_angles(x) for x in X]

# Load labels
Y = np.loadtxt("trainY.txt")
Y = torch.tensor(np.where(Y == 0, -1.0, 1.0), dtype=torch.float32)

# Split data into training and validation sets
np.random.seed(0)
num_data = len(X)
num_train = int(0.75 * num_data)
index = np.random.permutation(num_data)

# Convert to tensor
feats_train = torch.tensor([features[i] for i in index[:num_train]], dtype=torch.float32)
Y_train = Y[index[:num_train]]
feats_val = torch.tensor([features[i] for i in index[num_train:]], dtype=torch.float32)
Y_val = Y[index[num_train:]]


In [3]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Hyperparameters
input_size = 4  # number of input features after angle encoding
hidden_size = 10  # number of neurons in the hidden layer
output_size = 1  # binary classification
num_epochs = 100  # number of epochs
learning_rate = 0.001  # learning rate


In [4]:
# Initialize the model, loss function, and optimizer
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error for regression-like output
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    # Forward pass
    outputs = model(feats_train)
    loss = criterion(outputs.squeeze(), Y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.2261
Epoch [20/100], Loss: 1.1687
Epoch [30/100], Loss: 1.1263
Epoch [40/100], Loss: 1.0936
Epoch [50/100], Loss: 1.0678
Epoch [60/100], Loss: 1.0472
Epoch [70/100], Loss: 1.0314
Epoch [80/100], Loss: 1.0186
Epoch [90/100], Loss: 1.0094
Epoch [100/100], Loss: 1.0031


In [5]:
# Evaluate the model
model.eval()
with torch.no_grad():
    train_outputs = model(feats_train).squeeze()
    val_outputs = model(feats_val).squeeze()
    
    train_loss = criterion(train_outputs, Y_train)
    val_loss = criterion(val_outputs, Y_val)
    
    # Binary classification accuracy
    train_predictions = torch.where(train_outputs >= 0, 1.0, -1.0)
    val_predictions = torch.where(val_outputs >= 0, 1.0, -1.0)
    
    train_accuracy = (train_predictions == Y_train).float().mean()
    val_accuracy = (val_predictions == Y_val).float().mean()
    
    print(f'Training Loss: {train_loss.item():.4f}, Training Accuracy: {train_accuracy.item():.4f}')
    print(f'Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy.item():.4f}')

Training Loss: 1.0025, Training Accuracy: 0.4825
Validation Loss: 1.0121, Validation Accuracy: 0.4325
